In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Import Library**

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install --ignore-installed Pillow==9.0.0
!pip install pybind11
!pip install fastwer

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pytesseract
from pytesseract import Output
from PIL import Image
import re
from collections import Counter
import json
import fastwer
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# **Read Image**

In [ ]:
def read_img(destination_file_name):
    image = cv2.imread(destination_file_name)
    
    r = 800 / image.shape[0]
    dim = (int(image.shape[1] * r), 800)
    
    image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    
    return image

# **Pre-Processing Image**

In [ ]:
def preprocessing(image):
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  thresh = 127
  blackWhite = cv2.threshold(gray,thresh,255,cv2.THRESH_BINARY)[1]

  kernel = np.ones((3,3), np.uint8)
  img_erosion = cv2.erode(blackWhite, kernel, iterations=1)

  return img_erosion

# **Get Text from Image**

In [ ]:
def get_text(image_procesed):
  result = pytesseract.image_to_string(image_procesed)
  list_value = result.split("\n")

  return list_value

# **Typo Correction**

In [ ]:
path_kata_dasar = '/content/drive/MyDrive/Bangkit/katadasar.txt'

def words(text): return re.findall(r'\w+', text.lower())

def get_counter(path_kata_dasar):
  WORDS = Counter(words(open(path_kata_dasar).read()))
  return WORDS

WORDS = get_counter(path_kata_dasar)

def P(word, N=sum(get_counter(path_kata_dasar).values())):
    # "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    # "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    # "Generate possible spelling corrections for word."
    return (known([word], path_kata_dasar) or known(edits1(word), path_kata_dasar) or known(edits2(word), path_kata_dasar) or [word])

def known(words, path_kata_dasar):
    # "The subset of `words` that appear in the dictionary of WORDS."
    WORDS = get_counter(path_kata_dasar)
    return set(w for w in words if w in WORDS)

def edits1(word):
    # "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)] # [('', 'kemarin'), ('k', 'emarin'), ('ke', 'marin'), dst]
    deletes    = [L + R[1:]               for L, R in splits if R] # ['emarin', 'kmarin', 'kearin', dst]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1] # ['ekmarin', 'kmearin', 'keamrin', dst]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters] # ['aemarin', 'bemarin', 'cemarin', dst]
    inserts    = [L + c + R               for L, R in splits for c in letters] # ['akemarin', 'bkemarin', 'ckemarin', dst]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    # "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
def processing_word(list_value, WORDS):
  new_list = []
  for string in list_value:
    if string.isspace() or len(string)==0:
      continue
    lower_string = string.lower()
    no_punc_string = re.sub(r'[^\w\s]','', lower_string)
    no_wspace_string = no_punc_string.replace('\n', ' ')
    no_punc_string = re.sub(r'[^\w\s]','', lower_string)
    no_wspace_string = no_punc_string.replace('\n', ' ')
    no_wspace_string = re.sub(' +', ' ', no_wspace_string)
    new_list.append(correction(no_wspace_string))

  return new_list

In [ ]:
def correction_typo(new_list):
  new_value = []
  for string in new_list:
    temp = ''
    data = string.split(" ")
    for s in data:
      # print(correction(s))
      temp = temp + " " + correction(s)
    new_value.append(temp[1:])

  return new_value

In [ ]:
def get_kabupaten(new_value):
  kabupaten = new_value[1]
  kabupaten

  return kabupaten

# **Get Value**

In [ ]:
def txt_to_list(path):
  # opening the file in read mode
  my_file = open(path, "r")
    
  # reading the file
  data = my_file.read()
    
  # replacing end splitting the text 
  # when newline ('\n') is seen.
  data_into_list = data.split("\n")
  my_file.close()
  return data_into_list

In [ ]:
def get_pair(prevalue, new_value):
  pair = {}
  for string in new_value:
    temp = string.split(' ')
    new_temp = []
    flag = False
    keyFlag = False
    key = ''
    for word in temp:
      if flag == False and word in prevalue:
        flag = True
        keyFlag = True
        key = word
      if flag and keyFlag:
        pair[key] = ''
        keyFlag = False
      if flag and not keyFlag:
        pair[key] = pair[key] + " " + word

  return pair

In [ ]:
def processing_pair(pair, kabupaten):
  new_pair = {}
  for string in pair.keys():
    if string == 'kewarganegaraan' or string == 'agama' or string == 'hingga' or string == 'perkawinan':
      temp = pair[string].split(' ')
      temp = temp[2:3]
      new_pair[string] = ' '.join(temp)
    elif string == 'tempattgl':
      temp = pair[string].split(' ')
      temp = temp[2:]
      new_pair[string] = ' '.join(temp)
      new_pair[string] = new_pair[string].replace('lahir ', '')
    elif string == 'pekerjaan':
      temp = pair[string].split(' ')
      temp = temp[2:]
      # temp = ' '.join(temp)
      i = 0
      new_temp = []
      for t in temp:
        if i == len(temp)-1:
          break
        flag = False
        for d in t:
          if d.isdigit():
            flag = True
            break
        if flag:
          i = i+1
          if i == len(temp)-1: 
            break
        else:
          new_temp.append(t)
          i = i+1
          if i == len(temp)-1: 
            break
      new_pair[string] = ' '.join(new_temp)
    else:
      temp = pair[string].split(' ')
      temp = temp[2:]
      new_pair[string] = ' '.join(temp)
  new_pair['kabupaten'] = kabupaten

  return new_pair

In [ ]:
def get_blood_type(new_pair):
  darah = new_pair['kelamin'].split(' ')
  darah

  new_pair['kelamin'] = darah[0]
  new_pair['darah'] = darah[-1]

  return new_pair

# **Convert to JSON**

In [ ]:
def pair_to_JSON(new_pair):
  list_key = new_pair.keys()
  
  new_dict = {}

  if 'provinsi' in list_key:
    new_dict['province'] = new_pair['provinsi']

  if 'kabupaten' in list_key:
    new_dict['district'] = new_pair['kabupaten']

  if 'nik' in list_key:
    new_dict['id_number'] = new_pair['nik']

  if 'nama' in list_key:
    new_dict['name'] = new_pair['nama']

  if 'tempattgl' in list_key:
    new_dict['place_date_of_birth'] = new_pair['tempattgl']

  if 'kelamin' in list_key:
    new_dict['gender'] = new_pair['kelamin']

  if 'darah' in list_key:
    new_dict['blood_type'] = new_pair['darah']

  if 'keldesa' in list_key:
    new_dict['address'] = new_pair['keldesa']

  if 'rtrw' in list_key:
    new_dict['neighborhood'] = new_pair['rtrw']

  if 'keldesa' in list_key:
    new_dict['village'] = new_pair['keldesa']

  if 'kecamatan' in list_key:
    new_dict['subdistrict'] = new_pair['kecamatan']

  if 'agama' in list_key:
    new_dict['religion'] = new_pair['agama']

  if 'perkawinan' in list_key:
    new_dict['marital_status'] = new_pair['perkawinan']

  if 'pekerjaan' in list_key:
    new_dict['occupation'] = new_pair['pekerjaan']

  if 'kewarganegaraan' in list_key:
    new_dict['nationality'] = new_pair['kewarganegaraan']

  if 'hingga' in list_key:
    new_dict['expiry_date'] = new_pair['hingga']

  res = {}

  for key in new_dict.keys():
    res[key] = new_dict[key].upper()

  json_object = json.dumps(res, indent = 4)

  return json_object

# **Main**

In [ ]:
image_path = "/content/drive/MyDrive/Bangkit/21.jpg"

image = read_img(image_path)
img_proc = preprocessing(image)
list_text = get_text(img_proc)

path_kata_dasar = '/content/drive/MyDrive/Bangkit/katadasar.txt'
WORDS = get_counter(path_kata_dasar)

new_list = processing_word(list_text, WORDS)
new_value = correction_typo(new_list)
kabupaten = get_kabupaten(new_value)

prevalue = txt_to_list('/content/drive/MyDrive/Bangkit/key.txt')
pair = get_pair(prevalue, new_value)
new_pair = processing_pair(pair, kabupaten)

In [ ]:
new_pair = processing_pair(pair, kabupaten)
new_pair = get_blood_type(new_pair)
new_dict = pair_to_JSON(new_pair)
print(new_dict)

{
    "province": "JAWA TIMUR",
    "district": "O KOTA SURABAYA",
    "id_number": "3578090411850003",
    "name": "PRIMA GRAVIGARDIAN CHANDRA",
    "place_date_of_birth": "BURABAYA 04111985",
    "gender": "LAHIR",
    "blood_type": "O",
    "address": "SEMOLOWARU",
    "neighborhood": "006 006",
    "village": "SEMOLOWARU",
    "subdistrict": "SUKOLILO",
    "religion": "ISLAM",
    "marital_status": "BELUM",
    "occupation": "KARYAWAN SWASTA KOTA",
    "nationality": "WNI",
    "expiry_date": "04112018"
}


#**Evaluation Metrics-1** (KTP-24)

In [ ]:
output = pd.DataFrame(list(new_pair.items()), columns = ['information','value'])

In [ ]:
output['real_output'] = ''
output['cer'] = ''
output['wer'] = ''
output['word_similarity'] = ''

In [ ]:
output.loc[output['information'] == 'provinsi', 'real_output'] = "daerah istimewa yogyakarta"
output.loc[output['information'] == 'kabupaten', 'real_output'] = "kabupaten sleman"
output.loc[output['information'] == 'nik', 'real_output'] = "3471140209790001"
output.loc[output['information'] == 'nama', 'real_output'] = "riyanto se"
output.loc[output['information'] == 'tempattgl', 'real_output'] = "grobogan 02091979"
output.loc[output['information'] == 'kelamin', 'real_output'] = "lakilaki"
output.loc[output['information'] == 'darah', 'real_output'] = "o"
output.loc[output['information'] == 'alamat', 'real_output'] = "prm puri domas d 3 sempu"
output.loc[output['information'] == 'rtrw', 'real_output'] = "001 024"
output.loc[output['information'] == 'keldesa', 'real_output'] = "wedomartani"
output.loc[output['information'] == 'kecamatan', 'real_output'] = "ngemplak"
output.loc[output['information'] == 'agama', 'real_output'] = "islam"
output.loc[output['information'] == 'perkawinan', 'real_output'] = "kawin"
output.loc[output['information'] == 'pekerjaan', 'real_output'] = "pedagang"
output.loc[output['information'] == 'kewarganegaraan', 'real_output'] = "wni"
output.loc[output['information'] == 'hingga', 'real_output'] = "02092017"

output

,information,value,real_output,cer,wer,word_similarity
0,provinsi,daerah istimewa yogyakarta,daerah istimewa yogyakarta,,,
1,nik,3471140209790001,3471140209790001,,,
2,nama,rivanto e,riyanto se,,,
3,tempattgl,lam grobogan 02091979,grobogan 02091979,,,
4,kelamin,lakilaki,lakilaki,,,
5,rtrw,o 024,001 024,,,
6,keldesa,wedomartani o,wedomartani,,,
7,kecamatan,ngemplak,ngemplak,,,
8,agama,islam,islam,,,
9,perkawinan,kawin,kawin,,,


In [ ]:
for index, row in output.iterrows():
  cer = fastwer.score_sent(row['value'], row['real_output'], char_level=True)
  wer = fastwer.score_sent(row['value'], row['real_output'], char_level=False)

  value_list = word_tokenize(row['value'])
  real_list = word_tokenize(row['real_output'])

  value_set = set(value_list)
  real_set = set(real_list)

  l1 = []
  l2 = []

  vector_list = value_set.union(real_set) 
  for i in vector_list:
      if i in value_list: 
        l1.append(1) 
      else: 
        l1.append(0)
      if i in real_list: 
        l2.append(1)
      else: 
        l2.append(0)
  j = 0
  for i in range(len(vector_list)):
          j+= l1[i]*l2[i]
  
  division = float((sum(l1)*sum(l2))**0.5)
  if division != 0:
    word_similarity = j / division
  else:
    word_similarity = 0

  output.loc[output['information'] == row['information'], 'cer'] = round(cer,2) 
  output.loc[output['information'] == row['information'], 'wer'] = round(wer,2)
  output.loc[output['information'] == row['information'], 'word_similarity'] = round(word_similarity,2)

output

,information,value,real_output,cer,wer,word_similarity
0,provinsi,daerah istimewa yogyakarta,daerah istimewa yogyakarta,0.0,0.0,1.0
1,nik,3471140209790001,3471140209790001,0.0,0.0,1.0
2,nama,rivanto e,riyanto se,20.0,100.0,0.0
3,tempattgl,lam grobogan 02091979,grobogan 02091979,23.53,50.0,0.82
4,kelamin,lakilaki,lakilaki,0.0,0.0,1.0
5,rtrw,o 024,001 024,42.86,50.0,0.5
6,keldesa,wedomartani o,wedomartani,18.18,100.0,0.71
7,kecamatan,ngemplak,ngemplak,0.0,0.0,1.0
8,agama,islam,islam,0.0,0.0,1.0
9,perkawinan,kawin,kawin,0.0,0.0,1.0


In [ ]:
mean_cer1 = round(output['cer'].mean(),2)
mean_wer1 = round(output['wer'].mean(),2)
mean_similarity1 = round(output['word_similarity'].mean(),2)
print('mean CER is {}% \nmean WER is {}% \nmean word similarity is {}'.format(mean_cer1,mean_wer1,mean_similarity1))

mean CER is 22.65% 
mean WER is 42.86% 
mean word similarity is 0.77


#**Evaluation Metrics-2** (KTP-1)

In [ ]:
output = pd.DataFrame(list(new_pair.items()), columns = ['information','value'])

In [ ]:
output['real_output'] = ''
output['cer'] = ''
output['wer'] = ''
output['word_similarity'] = ''

In [ ]:
output.loc[output['information'] == 'provinsi', 'real_output'] = "jawa timur"
output.loc[output['information'] == 'kabupaten', 'real_output'] = "kabupaten kediri"
output.loc[output['information'] == 'nik', 'real_output'] = "3506042602660001"
output.loc[output['information'] == 'nama', 'real_output'] = "sulistyono"
output.loc[output['information'] == 'tempattgl', 'real_output'] = "kediri 26021966"
output.loc[output['information'] == 'kelamin', 'real_output'] = "lakilaki"
output.loc[output['information'] == 'darah', 'real_output'] = "-"
output.loc[output['information'] == 'alamat', 'real_output'] = "jl raya dsn purwokerto"
output.loc[output['information'] == 'rtrw', 'real_output'] = "002 003"
output.loc[output['information'] == 'keldesa', 'real_output'] = "purwokerto"
output.loc[output['information'] == 'kecamatan', 'real_output'] = "ngadiluwih"
output.loc[output['information'] == 'agama', 'real_output'] = "islam"
output.loc[output['information'] == 'perkawinan', 'real_output'] = "kawin"
output.loc[output['information'] == 'pekerjaan', 'real_output'] = "guru"
output.loc[output['information'] == 'kewarganegaraan', 'real_output'] = "wni"
output.loc[output['information'] == 'hingga', 'real_output'] = "26022017"

In [ ]:
for index, row in output.iterrows():
  cer = fastwer.score_sent(row['value'], row['real_output'], char_level=True)
  wer = fastwer.score_sent(row['value'], row['real_output'], char_level=False)

  value_list = word_tokenize(row['value'])
  real_list = word_tokenize(row['real_output'])

  value_set = set(value_list)
  real_set = set(real_list)

  l1 = []
  l2 = []

  vector_list = value_set.union(real_set) 
  for i in vector_list:
      if i in value_list: 
        l1.append(1) 
      else: 
        l1.append(0)
      if i in real_list: 
        l2.append(1)
      else: 
        l2.append(0)
  j = 0
  for i in range(len(vector_list)):
          j+= l1[i]*l2[i]
  
  division = float((sum(l1)*sum(l2))**0.5)
  if division != 0:
    word_similarity = j / division
  else:
    word_similarity = 0

  output.loc[output['information'] == row['information'], 'cer'] = round(cer,2) 
  output.loc[output['information'] == row['information'], 'wer'] = round(wer,2)
  output.loc[output['information'] == row['information'], 'word_similarity'] = round(word_similarity,2)

output

,information,value,real_output,cer,wer,word_similarity
0,provinsi,jawa timur,jawa timur,0.0,0.0,1.0
1,nik,350b042l02ll0001,3506042602660001,25.0,100.0,0.0
2,nama,sulistyono,sulistyono,0.0,0.0,1.0
3,tempattgl,kediri 26021966,kediri 26021966,0.0,0.0,1.0
4,kelamin,o,lakilaki,100.0,100.0,0.0
5,rtrw,002 003,002 003,0.0,0.0,1.0
6,pekerjaan,,guru,100.0,100.0,0
7,kewarganegaraan,wni,wni,0.0,0.0,1.0
8,hingga,26022017,26022017,0.0,0.0,1.0
9,kabupaten,kabupaten kediri,kabupaten kediri,0.0,0.0,1.0


In [ ]:
mean_cer2 = round(output['cer'].mean(),2)
mean_wer2 = round(output['wer'].mean(),2)
mean_similarity2 = round(output['word_similarity'].mean(),2)
print('mean CER is {}% \nmean WER is {}% \nmean word similarity is {}'.format(mean_cer2,mean_wer2,mean_similarity2))

mean CER is 29.55% 
mean WER is 36.36% 
mean word similarity is 0.64


#**Evaluation Metrics-3** (KTP-26)

In [ ]:
output = pd.DataFrame(list(new_pair.items()), columns = ['information','value'])

In [ ]:
output['real_output'] = ''
output['cer'] = ''
output['wer'] = ''
output['word_similarity'] = ''

In [ ]:
output.loc[output['information'] == 'provinsi', 'real_output'] = "dki jakarta"
output.loc[output['information'] == 'kabupaten', 'real_output'] = "jakarta barat"
output.loc[output['information'] == 'nik', 'real_output'] = "3173012512720017"
output.loc[output['information'] == 'nama', 'real_output'] = "antonius natanael k se"
output.loc[output['information'] == 'tempattgl', 'real_output'] = "gisting 25121972"
output.loc[output['information'] == 'kelamin', 'real_output'] = "lakilaki"
output.loc[output['information'] == 'darah', 'real_output'] = "o"
output.loc[output['information'] == 'alamat', 'real_output'] = "carina sayang blok z no 27"
output.loc[output['information'] == 'rtrw', 'real_output'] = "004 010"
output.loc[output['information'] == 'keldesa', 'real_output'] = "rawa buaya"
output.loc[output['information'] == 'kecamatan', 'real_output'] = "cengkareng"
output.loc[output['information'] == 'agama', 'real_output'] = "kristen"
output.loc[output['information'] == 'perkawinan', 'real_output'] = "kawin"
output.loc[output['information'] == 'pekerjaan', 'real_output'] = "karyawan swasta"
output.loc[output['information'] == 'kewarganegaraan', 'real_output'] = "wni"
output.loc[output['information'] == 'hingga', 'real_output'] = "25122016"

In [ ]:
for index, row in output.iterrows():
  cer = fastwer.score_sent(row['value'], row['real_output'], char_level=True)
  wer = fastwer.score_sent(row['value'], row['real_output'], char_level=False)

  value_list = word_tokenize(row['value'])
  real_list = word_tokenize(row['real_output'])

  value_set = set(value_list)
  real_set = set(real_list)

  l1 = []
  l2 = []

  vector_list = value_set.union(real_set) 
  for i in vector_list:
      if i in value_list: 
        l1.append(1) 
      else: 
        l1.append(0)
      if i in real_list: 
        l2.append(1)
      else: 
        l2.append(0)
  j = 0
  for i in range(len(vector_list)):
          j+= l1[i]*l2[i]
  
  division = float((sum(l1)*sum(l2))**0.5)
  if division != 0:
    word_similarity = j / division
  else:
    word_similarity = 0

  output.loc[output['information'] == row['information'], 'cer'] = round(cer,2) 
  output.loc[output['information'] == row['information'], 'wer'] = round(wer,2)
  output.loc[output['information'] == row['information'], 'word_similarity'] = round(word_similarity,2)

output

,information,value,real_output,cer,wer,word_similarity
0,provinsi,dki jakarta,dki jakarta,0.0,0.0,1.0
1,nik,3173012512720017,3173012512720017,0.0,0.0,1.0
2,nama,antonius natanael is,antonius natanael k se,13.64,50.0,0.58
3,tempattgl,gisting 25121972,gisting 25121972,0.0,0.0,1.0
4,kelamin,lakilaki,lakilaki,0.0,0.0,1.0
5,rtrw,008 010,004 010,14.29,50.0,0.5
6,keldesa,rawa buaya,rawa buaya,0.0,0.0,1.0
7,kecamatan,cengkareng,cengkareng,0.0,0.0,1.0
8,agama,kristen,kristen,0.0,0.0,1.0
9,perkawinan,kawin,kawin,0.0,0.0,1.0


In [ ]:
mean_cer3 = round(output['cer'].mean(),2)
mean_wer3 = round(output['wer'].mean(),2)
mean_similarity3 = round(output['word_similarity'].mean(),2)
print('mean CER is {}% \nmean WER is {}% \nmean word similarity is {}'.format(mean_cer3,mean_wer3,mean_similarity3))

mean CER is 1.86% 
mean WER is 6.67% 
mean word similarity is 0.94


#**Evaluation Metrics-4** (KTP-34)

In [ ]:
output = pd.DataFrame(list(new_pair.items()), columns = ['information','value'])
output

,information,value
0,provinsi,sumatera utara
1,nik,423010704750008
2,nama,jonson
3,kelamin,o
4,rtrw,00000
5,keldesa,ab kanopan
6,kecamatan,kualuh hulu
7,agama,budha
8,perkawinan,belum
9,pekerjaan,karyawan


In [ ]:
output['real_output'] = ''
output['cer'] = ''
output['wer'] = ''
output['word_similarity'] = ''

In [ ]:
output.loc[output['information'] == 'provinsi', 'real_output'] = "sumatera utara"
output.loc[output['information'] == 'kabupaten', 'real_output'] = "kabupaten labuhanbatu utara"
output.loc[output['information'] == 'nik', 'real_output'] = "1223010704750008"
output.loc[output['information'] == 'nama', 'real_output'] = "jonson"
output.loc[output['information'] == 'lahir', 'real_output'] = "b serbangan 07041975"
output.loc[output['information'] == 'kelamin', 'real_output'] = "lakilaki"
output.loc[output['information'] == 'darah', 'real_output'] = "-"
output.loc[output['information'] == 'alamat', 'real_output'] = "jl jend sudirman no 116"
output.loc[output['information'] == 'rtrw', 'real_output'] = "000 000"
output.loc[output['information'] == 'keldesa', 'real_output'] = "aek kanopan"
output.loc[output['information'] == 'kecamatan', 'real_output'] = "kualuh hulu"
output.loc[output['information'] == 'agama', 'real_output'] = "budha"
output.loc[output['information'] == 'perkawinan', 'real_output'] = "belum"
output.loc[output['information'] == 'pekerjaan', 'real_output'] = "karyawan swasta"
output.loc[output['information'] == 'kewarganegaraan', 'real_output'] = "wni"
output.loc[output['information'] == 'hingga', 'real_output'] = "seumur hidup"

In [ ]:
for index, row in output.iterrows():
  cer = fastwer.score_sent(row['value'], row['real_output'], char_level=True)
  wer = fastwer.score_sent(row['value'], row['real_output'], char_level=False)

  value_list = word_tokenize(row['value'])
  real_list = word_tokenize(row['real_output'])

  value_set = set(value_list)
  real_set = set(real_list)

  l1 = []
  l2 = []

  vector_list = value_set.union(real_set) 
  for i in vector_list:
      if i in value_list: 
        l1.append(1) 
      else: 
        l1.append(0)
      if i in real_list: 
        l2.append(1)
      else: 
        l2.append(0)
  j = 0
  for i in range(len(vector_list)):
          j+= l1[i]*l2[i]
  
  division = float((sum(l1)*sum(l2))**0.5)
  if division != 0:
    word_similarity = j / division
  else:
    word_similarity = 0

  output.loc[output['information'] == row['information'], 'cer'] = round(cer,2) 
  output.loc[output['information'] == row['information'], 'wer'] = round(wer,2)
  output.loc[output['information'] == row['information'], 'word_similarity'] = round(word_similarity,2)

output

,information,value,real_output,cer,wer,word_similarity
0,provinsi,sumatera utara,sumatera utara,0.0,0.0,1.0
1,nik,423010704750008,1223010704750008,12.5,100.0,0.0
2,nama,jonson,jonson,0.0,0.0,1.0
3,kelamin,o,lakilaki,100.0,100.0,0.0
4,rtrw,00000,000 000,28.57,100.0,0.0
5,keldesa,ab kanopan,aek kanopan,18.18,50.0,0.5
6,kecamatan,kualuh hulu,kualuh hulu,0.0,0.0,1.0
7,agama,budha,budha,0.0,0.0,1.0
8,perkawinan,belum,belum,0.0,0.0,1.0
9,pekerjaan,karyawan,karyawan swasta,46.67,50.0,0.71


In [ ]:
mean_cer4 = round(output['cer'].mean(),2)
mean_wer4 = round(output['wer'].mean(),2)
mean_similarity4 = round(output['word_similarity'].mean(),2)
print('mean CER is {}% \nmean WER is {}% \nmean word similarity is {}'.format(mean_cer4,mean_wer4,mean_similarity4))

mean CER is 25.42% 
mean WER is 39.29% 
mean word similarity is 0.64


#**Evaluation Metrics-5** (KTP-40)

In [ ]:
output = pd.DataFrame(list(new_pair.items()), columns = ['information','value'])
output

,information,value
0,provinsi,dki jakarta
1,nik,3173022707700008
2,nama,hartono
3,lahir,ab singkep 27071980
4,kelamin,lakilaki
5,rtrw,7004 005
6,keldesa,tomang
7,kecamatan,grogol petamburan
8,agama,budha
9,perkawinan,belum


In [ ]:
output['real_output'] = ''
output['cer'] = ''
output['wer'] = ''
output['word_similarity'] = ''

In [ ]:
output.loc[output['information'] == 'provinsi', 'real_output'] = "dki jakarta"
output.loc[output['information'] == 'kabupaten', 'real_output'] = "jakarta barat"
output.loc[output['information'] == 'nik', 'real_output'] = "3173022707900008"
output.loc[output['information'] == 'nama', 'real_output'] = "hartono"
output.loc[output['information'] == 'lahir', 'real_output'] = "dabo singkep 27071990"
output.loc[output['information'] == 'kelamin', 'real_output'] = "lakilaki"
output.loc[output['information'] == 'darah', 'real_output'] = "b"
output.loc[output['information'] == 'alamat', 'real_output'] = "jl pulo macam ix 91"
output.loc[output['information'] == 'rtrw', 'real_output'] = "004 005"
output.loc[output['information'] == 'keldesa', 'real_output'] = "tomang"
output.loc[output['information'] == 'kecamatan', 'real_output'] = "grogol petamburan"
output.loc[output['information'] == 'agama', 'real_output'] = "budha"
output.loc[output['information'] == 'perkawinan', 'real_output'] = "belum"
output.loc[output['information'] == 'pekerjaan', 'real_output'] = "pelajarmahasiswa"
output.loc[output['information'] == 'kewarganegaraan', 'real_output'] = "wni"
output.loc[output['information'] == 'hingga', 'real_output'] = "27072016"

In [ ]:
for index, row in output.iterrows():
  cer = fastwer.score_sent(row['value'], row['real_output'], char_level=True)
  wer = fastwer.score_sent(row['value'], row['real_output'], char_level=False)

  value_list = word_tokenize(row['value'])
  real_list = word_tokenize(row['real_output'])

  value_set = set(value_list)
  real_set = set(real_list)

  l1 = []
  l2 = []

  vector_list = value_set.union(real_set) 
  for i in vector_list:
      if i in value_list: 
        l1.append(1) 
      else: 
        l1.append(0)
      if i in real_list: 
        l2.append(1)
      else: 
        l2.append(0)
  j = 0
  for i in range(len(vector_list)):
          j+= l1[i]*l2[i]
  
  division = float((sum(l1)*sum(l2))**0.5)
  if division != 0:
    word_similarity = j / division
  else:
    word_similarity = 0

  output.loc[output['information'] == row['information'], 'cer'] = round(cer,2) 
  output.loc[output['information'] == row['information'], 'wer'] = round(wer,2)
  output.loc[output['information'] == row['information'], 'word_similarity'] = round(word_similarity,2)

output

,information,value,real_output,cer,wer,word_similarity
0,provinsi,dki jakarta,dki jakarta,0.0,0.0,1.0
1,nik,3173022707700008,3173022707900008,6.25,100.0,0.0
2,nama,hartono,hartono,0.0,0.0,1.0
3,lahir,ab singkep 27071980,dabo singkep 27071990,14.29,66.67,0.33
4,kelamin,lakilaki,lakilaki,0.0,0.0,1.0
5,rtrw,7004 005,004 005,14.29,50.0,0.5
6,keldesa,tomang,tomang,0.0,0.0,1.0
7,kecamatan,grogol petamburan,grogol petamburan,0.0,0.0,1.0
8,agama,budha,budha,0.0,0.0,1.0
9,perkawinan,belum,belum,0.0,0.0,1.0


In [ ]:
mean_cer5 = round(output['cer'].mean(),2)
mean_wer5 = round(output['wer'].mean(),2)
mean_similarity5 = round(output['word_similarity'].mean(),2)
print('mean CER is {}% \nmean WER is {}% \nmean word similarity is {}'.format(mean_cer5,mean_wer5,mean_similarity5))

mean CER is 5.66% 
mean WER is 21.11% 
mean word similarity is 0.84


#**Evaluation Metrics-6** (KTP-27)

In [ ]:
output = pd.DataFrame(list(new_pair.items()), columns = ['information','value'])
output

,information,value
0,provinsi,jawa tengah
1,nik,3310152505650001
2,nama,suharyanto
3,tempattgl,klaten 25051965
4,kelamin,lakilaki
5,rtrw,003 o 003
6,keldesa,80to
7,kecamatan,wonosari
8,agama,islam
9,perkawinan,kawin


In [ ]:
output['real_output'] = ''
output['cer'] = ''
output['wer'] = ''
output['word_similarity'] = ''

In [ ]:
output.loc[output['information'] == 'provinsi', 'real_output'] = "jawa tengah"
output.loc[output['information'] == 'kabupaten', 'real_output'] = "kabupaten klaten"
output.loc[output['information'] == 'nik', 'real_output'] = "3310152505650001"
output.loc[output['information'] == 'nama', 'real_output'] = "suharyanto"
output.loc[output['information'] == 'tempattgl', 'real_output'] = "klaten 25051965"
output.loc[output['information'] == 'kelamin', 'real_output'] = "lakilaki"
output.loc[output['information'] == 'darah', 'real_output'] = "o"
output.loc[output['information'] == 'alamat', 'real_output'] = "bakalan"
output.loc[output['information'] == 'rtrw', 'real_output'] = "003 003"
output.loc[output['information'] == 'keldesa', 'real_output'] = "boto"
output.loc[output['information'] == 'kecamatan', 'real_output'] = "wonosari"
output.loc[output['information'] == 'agama', 'real_output'] = "islam"
output.loc[output['information'] == 'perkawinan', 'real_output'] = "kawin"
output.loc[output['information'] == 'pekerjaan', 'real_output'] = "karyawan swasta"
output.loc[output['information'] == 'kewarganegaraan', 'real_output'] = "wni"
output.loc[output['information'] == 'hingga', 'real_output'] = "25052017"

In [ ]:
for index, row in output.iterrows():
  cer = fastwer.score_sent(row['value'], row['real_output'], char_level=True)
  wer = fastwer.score_sent(row['value'], row['real_output'], char_level=False)

  value_list = word_tokenize(row['value'])
  real_list = word_tokenize(row['real_output'])

  value_set = set(value_list)
  real_set = set(real_list)

  l1 = []
  l2 = []

  vector_list = value_set.union(real_set) 
  for i in vector_list:
      if i in value_list: 
        l1.append(1) 
      else: 
        l1.append(0)
      if i in real_list: 
        l2.append(1)
      else: 
        l2.append(0)
  j = 0
  for i in range(len(vector_list)):
          j+= l1[i]*l2[i]
  
  division = float((sum(l1)*sum(l2))**0.5)
  if division != 0:
    word_similarity = j / division
  else:
    word_similarity = 0

  output.loc[output['information'] == row['information'], 'cer'] = round(cer,2) 
  output.loc[output['information'] == row['information'], 'wer'] = round(wer,2)
  output.loc[output['information'] == row['information'], 'word_similarity'] = round(word_similarity,2)

output

,information,value,real_output,cer,wer,word_similarity
0,provinsi,jawa tengah,jawa tengah,0.0,0.0,1.0
1,nik,3310152505650001,3310152505650001,0.0,0.0,1.0
2,nama,suharyanto,suharyanto,0.0,0.0,1.0
3,tempattgl,klaten 25051965,klaten 25051965,0.0,0.0,1.0
4,kelamin,lakilaki,lakilaki,0.0,0.0,1.0
5,rtrw,003 o 003,003 003,28.57,50.0,0.71
6,keldesa,80to,boto,50.0,100.0,0.0
7,kecamatan,wonosari,wonosari,0.0,0.0,1.0
8,agama,islam,islam,0.0,0.0,1.0
9,perkawinan,kawin,kawin,0.0,0.0,1.0


In [ ]:
mean_cer6 = round(output['cer'].mean(),2)
mean_wer6 = round(output['wer'].mean(),2)
mean_similarity6 = round(output['word_similarity'].mean(),2)
print('mean CER is {}% \nmean WER is {}% \nmean word similarity is {}'.format(mean_cer6,mean_wer6,mean_similarity6))

mean CER is 5.24% 
mean WER is 10.0% 
mean word similarity is 0.91


#**Evaluation Metrics-7** (KTP-21)

In [ ]:
output = pd.DataFrame(list(new_pair.items()), columns = ['information','value'])
output

,information,value
0,provinsi,jawa timur
1,nik,3578090411850003
2,nama,prima gravigardian chandra
3,tempattgl,burabaya 04111985
4,kelamin,lahir
5,rtrw,006 006
6,keldesa,semolowaru
7,kecamatan,sukolilo
8,agama,islam
9,perkawinan,belum


In [ ]:
output['real_output'] = ''
output['cer'] = ''
output['wer'] = ''
output['word_similarity'] = ''

In [ ]:
output.loc[output['information'] == 'provinsi', 'real_output'] = "jawa timur"
output.loc[output['information'] == 'kabupaten', 'real_output'] = "kota surabaya"
output.loc[output['information'] == 'nik', 'real_output'] = "3578090411850003"
output.loc[output['information'] == 'nama', 'real_output'] = "prima gravigardian chandra"
output.loc[output['information'] == 'tempattgl', 'real_output'] = "surabaya 04111985"
output.loc[output['information'] == 'kelamin', 'real_output'] = "lakilaki"
output.loc[output['information'] == 'darah', 'real_output'] = "o"
output.loc[output['information'] == 'alamat', 'real_output'] = "semolowaru elok blok a6"
output.loc[output['information'] == 'rtrw', 'real_output'] = "006 006"
output.loc[output['information'] == 'keldesa', 'real_output'] = "semolowaru"
output.loc[output['information'] == 'kecamatan', 'real_output'] = "sukolilo"
output.loc[output['information'] == 'agama', 'real_output'] = "islam"
output.loc[output['information'] == 'perkawinan', 'real_output'] = "belum"
output.loc[output['information'] == 'pekerjaan', 'real_output'] = "karyawan swasta"
output.loc[output['information'] == 'kewarganegaraan', 'real_output'] = "wni"
output.loc[output['information'] == 'hingga', 'real_output'] = "04112018"

In [ ]:
for index, row in output.iterrows():
  cer = fastwer.score_sent(row['value'], row['real_output'], char_level=True)
  wer = fastwer.score_sent(row['value'], row['real_output'], char_level=False)

  value_list = word_tokenize(row['value'])
  real_list = word_tokenize(row['real_output'])

  value_set = set(value_list)
  real_set = set(real_list)

  l1 = []
  l2 = []

  vector_list = value_set.union(real_set) 
  for i in vector_list:
      if i in value_list: 
        l1.append(1) 
      else: 
        l1.append(0)
      if i in real_list: 
        l2.append(1)
      else: 
        l2.append(0)
  j = 0
  for i in range(len(vector_list)):
          j+= l1[i]*l2[i]
  
  division = float((sum(l1)*sum(l2))**0.5)
  if division != 0:
    word_similarity = j / division
  else:
    word_similarity = 0

  output.loc[output['information'] == row['information'], 'cer'] = round(cer,2) 
  output.loc[output['information'] == row['information'], 'wer'] = round(wer,2)
  output.loc[output['information'] == row['information'], 'word_similarity'] = round(word_similarity,2)

output

,information,value,real_output,cer,wer,word_similarity
0,provinsi,jawa timur,jawa timur,0.0,0.0,1.0
1,nik,3578090411850003,3578090411850003,0.0,0.0,1.0
2,nama,prima gravigardian chandra,prima gravigardian chandra,0.0,0.0,1.0
3,tempattgl,burabaya 04111985,surabaya 04111985,5.88,50.0,0.5
4,kelamin,lahir,lakilaki,62.5,100.0,0.0
5,rtrw,006 006,006 006,0.0,0.0,1.0
6,keldesa,semolowaru,semolowaru,0.0,0.0,1.0
7,kecamatan,sukolilo,sukolilo,0.0,0.0,1.0
8,agama,islam,islam,0.0,0.0,1.0
9,perkawinan,belum,belum,0.0,0.0,1.0


In [ ]:
mean_cer7 = round(output['cer'].mean(),2)
mean_wer7 = round(output['wer'].mean(),2)
mean_similarity7 = round(output['word_similarity'].mean(),2)
print('mean CER is {}% \nmean WER is {}% \nmean word similarity is {}'.format(mean_cer7,mean_wer7,mean_similarity7))

mean CER is 7.81% 
mean WER is 16.67% 
mean word similarity is 0.88


#**Conclusion**

In [ ]:
cer = round((mean_cer1 + mean_cer2 + mean_cer3 + mean_cer4 + mean_cer5 + mean_cer6 + mean_cer7)/7,2)
wer = round((mean_wer1 + mean_wer2 + mean_wer3 + mean_wer4 + mean_wer5 + mean_cer6 + mean_wer7)/7,2)
similarity = round((mean_similarity1 + mean_similarity2 + mean_similarity3 + mean_similarity4 +mean_similarity5 + mean_similarity6 + mean_similarity7)/7*100,2)

In [ ]:
print('mean CER is {}% \nmean WER is {}% \nmean_similarity is {}%'.format(cer,wer,similarity))

mean CER is 14.03% 
mean WER is 24.03% 
mean_similarity is 80.29%
